# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

## Install Modules

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 624.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 13.2 MB/s eta 0:00:00


In [ ]:
!pip install docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 7.1 MB/s eta 0:00:00


## Initialization

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving OutdoorClothingCatalog_1000.csv to OutdoorClothingCatalog_1000.csv


In [ ]:
# Initialize document loader for CSV files
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [ ]:
# Set up index creation for querying
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
# Build a document index using embeddings for efficient retrieval
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=OpenAIEmbeddings()
).from_loaders([loader])

AuthenticationError: Error code: 401 - {'error': {'message': "Authentication Error, API Key not allowed to access model. This token can only access models=['qwen2.5']. Tried to access text-embedding-ada-002", 'type': 'auth_error', 'param': 'None', 'code': '401'}}

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [ ]:
# Execute the query and display response
response = index.query(query)

In [ ]:
display(Markdown(response))

## Step By Step

In [ ]:
# Reload the CSV and inspect a sample entry
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

In [ ]:
# Initialize embedding function
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [ ]:
# Print embedding details for verification
print(len(embed))

In [ ]:
print(embed[:5])

In [ ]:
# Build an in-memory search database for document retrieval
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

In [ ]:
# Execute a similarity-based search query
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
# Display the first retrieved document and count of retrieved items
len(docs)

In [ ]:
docs[0]

In [ ]:
# Initialize retriever object for use in LLM-based QA
retriever = db.as_retriever()

In [ ]:
# Aggregate content from retrieved documents
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [ ]:
# Set up language model with custom settings
llm = ChatOpenAI(model="qwen2.5",
                 api_key="",
                 base_url='',
                 temperature=0.0)

In [ ]:
# Run the LLM with the retrieved documents and display results in markdown format
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

In [ ]:
display(Markdown(response))

In [ ]:
# Create a LangChain QA system with the "stuff" chain type for efficient document handling
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [ ]:
# Run a structured query through the QA system
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
# Example of running the query directly on the index for comparison
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
# Rebuild index with specified embeddings (re-run if document list updates)
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

Reminder: Download your notebook to you local computer to save your work.